In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd

df_train = pd.read_csv('train_kaggle.csv')
df_train

,Id,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,1
9,9,0


In [3]:
import pandas as pd
def load_features(col):
    return pd.read_parquet("filtered_features/filtered_feature{}.gzip".format(col))

In [4]:
import pandas as pd

def load_test_features(col):
    return pd.read_parquet("filtered_test_features/filtered_test_feature{}.gzip".format(col))

In [5]:
NUM_OF_COL = 40

In [6]:
SkipCol = {}
SkipCol[18] = True
SkipCol

{18: True}

In [7]:
y = df_train['label']
y = y.values
y

array([0, 0, 0, ..., 1, 0, 1], dtype=int64)

In [8]:
df_test = pd.read_csv('sample_solution.csv')


In [9]:
y.shape

(30336,)

In [10]:
import numpy as np
XTrain = np.load('XTrain.npy', allow_pickle=True)
XTrain.shape

(30336, 3248)

In [11]:
XTest = np.load('XTest.npy', allow_pickle=True)
XTest.shape

(10000, 3248)

In [40]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(XTrain, y, test_size=0.2, random_state=42)

In [41]:
import lightgbm as lgb

train_set = lgb.Dataset(X_train, y_train)
valid_set = lgb.Dataset(X_valid, y_valid)

In [62]:
param = {'num_leaves': 31, 'objective': 'binary'}
param['metric'] = 'auc'
bst = lgb.train(param, train_set, 5, valid_sets=valid_set, early_stopping_rounds=5)
bst.best_iteration 

[1]	valid_0's auc: 0.852927
Training until validation scores don't improve for 5 rounds
[2]	valid_0's auc: 0.8823
[3]	valid_0's auc: 0.904984
[4]	valid_0's auc: 0.915694
[5]	valid_0's auc: 0.924469
Did not meet early stopping. Best iteration is:
[5]	valid_0's auc: 0.924469


5

In [67]:
bst.best_score['valid_0']['auc']

0.9244686197253724

In [72]:
import lightgbm as lgb
from hyperopt import STATUS_OK

ITER = 50
STOP_ROUND = 5

# Create the dataset
def objective(params, n_folds = N_FOLDS):
    """Objective function for Gradient Boosting Machine Hyperparameter Tuning"""
    params['num_leaves'] = int(params['num_leaves'])
    params['subsample_for_bin'] = int(params['subsample_for_bin'])
    params['min_child_samples'] = int(params['min_child_samples'])
    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evalute based on ROC AUC
    bst = lgb.train(params, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)
    bst.save_model('model.txt', num_iteration=bst.best_iteration)
  
    # Extract the best score
    best_score = bst.best_score['valid_0']['auc']
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

In [73]:
from hyperopt import hp

# Define the search space
space = {
    'boosting_type': hp.choice('boosting_type', 
                                [
                                    'gbdt',
                                    'dart',
                                    'goss'
                                ]),
    'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    'metric': 'auc'
}

space

{'boosting_type': <hyperopt.pyll.base.Apply at 0x2168303ebe0>,
 'num_leaves': <hyperopt.pyll.base.Apply at 0x2168303ea90>,
 'learning_rate': <hyperopt.pyll.base.Apply at 0x2168303e8d0>,
 'subsample_for_bin': <hyperopt.pyll.base.Apply at 0x2168303ecc0>,
 'min_child_samples': <hyperopt.pyll.base.Apply at 0x21682197668>,
 'reg_alpha': <hyperopt.pyll.base.Apply at 0x2168301d128>,
 'reg_lambda': <hyperopt.pyll.base.Apply at 0x2168301d278>,
 'colsample_bytree': <hyperopt.pyll.base.Apply at 0x2168301d3c8>,
 'metric': 'auc'}

In [74]:
from hyperopt import Trials
# Trials object to track progress
bayes_trials = Trials()

In [75]:
from hyperopt import fmin
from hyperopt import tpe

MAX_EVALS = 500

# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials)
best

[1]	valid_0's auc: 0.907715                                                                   
Training until validation scores don't improve for 5 rounds                                   
[2]	valid_0's auc: 0.916245                                                                   
[3]	valid_0's auc: 0.925285                                                                   
[4]	valid_0's auc: 0.926795                                                                   
[5]	valid_0's auc: 0.929827                                                                   
[6]	valid_0's auc: 0.93199                                                                    
[7]	valid_0's auc: 0.932494                                                                   
[8]	valid_0's auc: 0.933407                                                                   
[9]	valid_0's auc: 0.93319                                                                    
[10]	valid_0's auc: 0.93449                       

[10]	valid_0's auc: 0.932675                                                                  
[11]	valid_0's auc: 0.93414                                                                   
[12]	valid_0's auc: 0.933895                                                                  
[13]	valid_0's auc: 0.934478                                                                  
[14]	valid_0's auc: 0.935842                                                                  
[15]	valid_0's auc: 0.936667                                                                  
[16]	valid_0's auc: 0.937841                                                                  
[17]	valid_0's auc: 0.938615                                                                  
[18]	valid_0's auc: 0.938481                                                                  
[19]	valid_0's auc: 0.939731                                                                  
[20]	valid_0's auc: 0.939718                      

[20]	valid_0's auc: 0.935163                                                                  
[21]	valid_0's auc: 0.934783                                                                  
[22]	valid_0's auc: 0.935007                                                                  
[23]	valid_0's auc: 0.934914                                                                  
[24]	valid_0's auc: 0.935486                                                                  
[25]	valid_0's auc: 0.935806                                                                  
[26]	valid_0's auc: 0.935742                                                                  
[27]	valid_0's auc: 0.936139                                                                  
[28]	valid_0's auc: 0.936338                                                                  
[29]	valid_0's auc: 0.936797                                                                  
[30]	valid_0's auc: 0.936943                      

[5]	valid_0's auc: 0.924919                                                                   
[6]	valid_0's auc: 0.927016                                                                   
[7]	valid_0's auc: 0.92754                                                                    
[8]	valid_0's auc: 0.928106                                                                   
[9]	valid_0's auc: 0.928359                                                                   
[10]	valid_0's auc: 0.930078                                                                  
[11]	valid_0's auc: 0.931173                                                                  
[12]	valid_0's auc: 0.930771                                                                  
[13]	valid_0's auc: 0.931992                                                                  
[14]	valid_0's auc: 0.932402                                                                  
[15]	valid_0's auc: 0.933122                      

[8]	valid_0's auc: 0.939045                                                                   
[9]	valid_0's auc: 0.941573                                                                   
[10]	valid_0's auc: 0.941193                                                                  
[11]	valid_0's auc: 0.941413                                                                  
[12]	valid_0's auc: 0.942015                                                                  
[13]	valid_0's auc: 0.944143                                                                  
[14]	valid_0's auc: 0.942407                                                                  
[15]	valid_0's auc: 0.943731                                                                  
[16]	valid_0's auc: 0.945283                                                                  
[17]	valid_0's auc: 0.94482                                                                   
[18]	valid_0's auc: 0.944406                      

...

[21]	valid_0's auc: 0.93769                                                                   
[22]	valid_0's auc: 0.938536                                                                  
[23]	valid_0's auc: 0.938807                                                                  
[24]	valid_0's auc: 0.938696                                                                  
[25]	valid_0's auc: 0.939768                                                                  
[26]	valid_0's auc: 0.940232                                                                  
[27]	valid_0's auc: 0.94094                                                                   
[28]	valid_0's auc: 0.940618                                                                  
[29]	valid_0's auc: 0.941456                                                                  
[30]	valid_0's auc: 0.941984                                                                  
[31]	valid_0's auc: 0.942708                      

[25]	valid_0's auc: 0.944042                                                                  
[26]	valid_0's auc: 0.944169                                                                  
[27]	valid_0's auc: 0.944718                                                                  
[28]	valid_0's auc: 0.94508                                                                   
[29]	valid_0's auc: 0.945577                                                                  
[30]	valid_0's auc: 0.945735                                                                  
[31]	valid_0's auc: 0.945934                                                                  
[32]	valid_0's auc: 0.94623                                                                   
[33]	valid_0's auc: 0.946134                                                                  
[34]	valid_0's auc: 0.946446                                                                  
[35]	valid_0's auc: 0.946486                      

[10]	valid_0's auc: 0.934737                                                                  
[11]	valid_0's auc: 0.935448                                                                  
[12]	valid_0's auc: 0.935445                                                                  
[13]	valid_0's auc: 0.93636                                                                   
[14]	valid_0's auc: 0.886341                                                                  
[15]	valid_0's auc: 0.438814                                                                  
[16]	valid_0's auc: 0.84443                                                                   
[17]	valid_0's auc: 0.378538                                                                  
[18]	valid_0's auc: 0.672535                                                                  
Early stopping, best iteration is:                                                            
[13]	valid_0's auc: 0.93636
[1]	valid_0's auc: 0.9

[25]	valid_0's auc: 0.951723                                                                  
[26]	valid_0's auc: 0.951655                                                                  
[27]	valid_0's auc: 0.951448                                                                  
[28]	valid_0's auc: 0.951624                                                                  
[29]	valid_0's auc: 0.951222                                                                  
[30]	valid_0's auc: 0.950996                                                                  
Early stopping, best iteration is:                                                            
[25]	valid_0's auc: 0.951723
[1]	valid_0's auc: 0.886335                                                                   
Training until validation scores don't improve for 5 rounds                                   
[2]	valid_0's auc: 0.900492                                                                   
[3]	valid_0's auc: 0.

{'boosting_type': 1,
 'colsample_by_tree': 0.7912493015275733,
 'learning_rate': 0.1825037992404684,
 'min_child_samples': 95.0,
 'num_leaves': 48.0,
 'reg_alpha': 0.5753421176471192,
 'reg_lambda': 0.6231841891673146,
 'subsample_for_bin': 20000.0}

In [78]:
best['boosting_type'] = 'dart'
best['num_leaves'] = int(best['num_leaves'])
best['subsample_for_bin'] = int(best['subsample_for_bin'])
best['min_child_samples'] = int(best['min_child_samples'])
bestModel = lgb.train(best, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)

In [79]:
bestModel.save_model('model.txt', num_iteration=bestModel.best_iteration)

In [81]:
bestModel.num_trees()

50

In [83]:
best

{'boosting_type': 'dart',
 'colsample_by_tree': 0.7912493015275733,
 'learning_rate': 0.1825037992404684,
 'min_child_samples': 95,
 'num_leaves': 48,
 'reg_alpha': 0.5753421176471192,
 'reg_lambda': 0.6231841891673146,
 'subsample_for_bin': 20000}

In [95]:
from lightgbm import LGBMClassifier

gb_clf = LGBMClassifier(boosting_type=best['boosting_type'],
                        num_leaves=best['num_leaves'],
                        learning_rate=best['learning_rate'],
                        subsample_for_bin=best['subsample_for_bin'],
                        min_child_samples=best['min_child_samples'],
                        reg_alpha=best['reg_alpha'],
                        reg_lambda=best['reg_lambda'],
                        colsample_bytree=best['colsample_by_tree'])
gb_clf.fit(XTrain, y)

LGBMClassifier(boosting_type='dart', class_weight=None,
        colsample_bytree=0.7912493015275733, importance_type='split',
        learning_rate=0.1825037992404684, max_depth=-1,
        min_child_samples=95, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=48, objective=None,
        random_state=None, reg_alpha=0.5753421176471192,
        reg_lambda=0.6231841891673146, silent=True, subsample=1.0,
        subsample_for_bin=20000, subsample_freq=0)

In [96]:
probs = gb_clf.predict_proba(XTest)
YTest = probs[:, 0]
YTest

array([0.99113728, 0.99650633, 0.98317971, ..., 0.99526574, 0.98510779,
       0.99783813])

In [97]:
np.max(YTest)

0.9992250705129174

In [98]:
df_test['Predicted'] = YTest
df_test

,Id,Predicted
0,0,0.991137
1,1,0.996506
2,2,0.983180
3,3,0.994090
4,4,0.992885
5,5,0.989909
6,6,0.991144
7,7,0.834834
8,8,0.996647
9,9,0.994923


In [99]:
df_test.to_csv('test.csv', index=False)

In [ ]:
from lightgbm import LGBMModel
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

clf = LGBMModel()
parameters = {'learning_rate': sp_randFloat(),
                  'subsample'    : sp_randFloat(),
                  'n_estimators' : sp_randInt(100, 1000),
                  'max_depth'    : sp_randInt(4, 10) 
                 }

randm = RandomizedSearchCV(estimator=gb_clf, param_distributions = parameters, 
                           cv = 5, n_iter = 10, n_jobs=-1)
randm.fit(XTrain, y)

# Results from Random Search
print("\n========================================================")
print(" Results from Random Search " )
print("========================================================")    

print("\n The best estimator across ALL searched params:\n",
      randm.best_estimator_)

print("\n The best score across ALL searched params:\n",
      randm.best_score_)

print("\n The best parameters across ALL searched params:\n",
      randm.best_params_)

print("\n ========================================================")

gb_clf = randm.best_estimator_